<a href="https://colab.research.google.com/github/illiadil/segformer_hpo/blob/main/experiment_segformer_hpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/illiadil/segformer_hpo

fatal: destination path 'segformer_hpo' already exists and is not an empty directory.


In [2]:
cd segformer_hpo

/content/segformer_hpo


In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
drive.mount('/content/drive/')
from sklearn.model_selection import train_test_split
from skimage.io import imread, imshow
from skimage.color import rgba2rgb
from models import SegFormer_B0

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
IMG_WIDTH = 640
IMG_HEIGHT = 480
IMG_CHANNELS = 3
DATASET_SIZE = 200

X = np.zeros((DATASET_SIZE, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y = np.zeros((DATASET_SIZE, IMG_HEIGHT, IMG_WIDTH, 1), dtype=float)

for i in range(1,DATASET_SIZE):
  image = imread("/content/drive/MyDrive/PhD/label_tool/dataset/images/"+str(i).zfill(5)+".png")[:,:,:3]
  mask_ = imread("/content/drive/MyDrive/PhD/label_tool/dataset/masks/"+str(i).zfill(5)+"_mask.png")
  X[i] = image
  mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)
  mask_ = np.expand_dims(mask_, axis=-1)
  mask = np.maximum(mask, mask_[:,:,1])
  y[i] = mask / 255

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [6]:
lr_schedule_exponential = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)

lr_schedule_polynomial = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    end_learning_rate=0.0001,
    power=1.0,
    cycle=False,
    name=None,
)

In [7]:
adam_exponential = tf.keras.optimizers.experimental.Adam()#lr_schedule_exponential)
rmsprop_exponential = tf.keras.optimizers.experimental.RMSprop()#lr_schedule_exponential)
sgd_exponential = tf.keras.optimizers.experimental.SGD()#lr_schedule_exponential)

adam_polynomial = tf.keras.optimizers.experimental.Adam()#lr_schedule_polynomial)
rmsprop_polynomial = tf.keras.optimizers.experimental.RMSprop()#lr_schedule_polynomial)
sgd_polynomial = tf.keras.optimizers.experimental.SGD()#lr_schedule_polynomial)

In [8]:
def iou(y_true, y_pred):
  threshold = 0.8
  y_prediction = y_pred[:,:,1] > threshold
  y_ground_truth = y_true[:,:,0] > 0.5
  TP = (y_prediction == y_ground_truth)
  FN_and_FP = (y_ground_truth != y_prediction)
  TP_sum = float(tf.reduce_sum(tf.cast(TP, tf.float32)))
  FN_and_FP_sum = float(tf.reduce_sum(tf.cast(FN_and_FP, tf.float32)))
  return TP_sum/( TP_sum + FN_and_FP_sum)

In [9]:
def calculate_miou(mdl):
  threshold = 0.8
  total_iou = 0
  y_predictions = mdl.predict(X_test)
  for i in range(X_test.shape[0]):
    y_ground_truth = y_test[i]
    y_prediction = np.argmax(y_predictions[i], axis=-1)
    TP = (y_prediction == y_ground_truth[:,:,0])
    FN_and_FP = (y_ground_truth[:,:,0] != y_prediction)
    iou = TP.sum()/( TP.sum() + FN_and_FP.sum() )
    total_iou = total_iou + iou
  average_iou = total_iou / X_test.shape[0]*100
  return "{:.2f}".format(average_iou)

In [10]:
from keras.optimizers.legacy import Adam, SGD, RMSprop
from keras.callbacks import LearningRateScheduler
import math

# Define initial learning rate
initial_learning_rate = 0.01
# Define decay steps
decay_steps = 15
# Define decay rate
decay_rate = 0.1

# Define learning rate decay function
def lr_time_based_decay_linear(epoch, lr):
    return lr * 1 / (1 + decay_rate * epoch / decay_steps)
def lr_time_based_decay_exponential(epoch, lr):
    return lr * math.exp( decay_rate * epoch / decay_steps)
decays = [lr_time_based_decay_linear,lr_time_based_decay_exponential]
decays_name = ["Linear","Exponential"]

# Initialize Adam optimizer with learning rate decay
adam_optimizer = Adam(learning_rate=initial_learning_rate)
sgd_optimizer = SGD(learning_rate=initial_learning_rate)
rmsprop_optimizer = RMSprop(learning_rate=initial_learning_rate)

optimizers = [adam_optimizer,rmsprop_optimizer,sgd_optimizer]
optimizers_name = ["Adam","RMSprop","SGD"]
miou_values = []

for j in range(0,2):
  for i in range(3):
    model = SegFormer_B0(input_shape = (480, 640, 3), num_classes = 2)
    model.compile(optimizer=optimizers[i], loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_val,y_val),batch_size=16, epochs=15, callbacks=[LearningRateScheduler(decays[j], verbose=0)])
    model.save_weights("segformer_b0_"+optimizers_name[i]+"_"+decays_name[j]+".weights.h5")
    miou = calculate_miou(model)
    miou_values.append(miou)
    print(decays_name[j]+" : " +optimizers_name[i]+ " -> mIoU : " + miou + "%")

Epoch 1/15
9/9 [==============================] - 44s 2s/step - loss: nan - accuracy: 0.6799 - val_loss: nan - val_accuracy: 0.8190 - lr: 0.0100
Epoch 2/15
9/9 [==============================] - 8s 896ms/step - loss: nan - accuracy: 0.7093 - val_loss: nan - val_accuracy: 0.8190 - lr: 0.0099
Epoch 3/15
9/9 [==============================] - 7s 827ms/step - loss: nan - accuracy: 0.7093 - val_loss: nan - val_accuracy: 0.8190 - lr: 0.0098
Epoch 4/15
9/9 [==============================] - 8s 905ms/step - loss: nan - accuracy: 0.7093 - val_loss: nan - val_accuracy: 0.8190 - lr: 0.0096
Epoch 5/15
9/9 [==============================] - 8s 909ms/step - loss: nan - accuracy: 0.7093 - val_loss: nan - val_accuracy: 0.8190 - lr: 0.0094
Epoch 6/15
9/9 [==============================] - 8s 908ms/step - loss: nan - accuracy: 0.7093 - val_loss: nan - val_accuracy: 0.8190 - lr: 0.0091
Epoch 7/15
9/9 [==============================] - 7s 837ms/step - loss: nan - accuracy: 0.7093 - val_loss: nan - val_acc

2/2 [==============================] - 4s 168ms/step
Exponential : RMSprop -> mIoU : 73.52%
Epoch 1/15
9/9 [==============================] - 31s 2s/step - loss: 0.7567 - accuracy: 0.5398 - val_loss: 0.6264 - val_accuracy: 0.8002 - lr: 0.0051
Epoch 2/15
9/9 [==============================] - 8s 942ms/step - loss: 0.6378 - accuracy: 0.6470 - val_loss: 0.6083 - val_accuracy: 0.8111 - lr: 0.0051
Epoch 3/15
9/9 [==============================] - 8s 884ms/step - loss: 0.5339 - accuracy: 0.7232 - val_loss: 0.8720 - val_accuracy: 0.1937 - lr: 0.0052
Epoch 4/15
9/9 [==============================] - 8s 956ms/step - loss: 0.5094 - accuracy: 0.7450 - val_loss: 0.4412 - val_accuracy: 0.8163 - lr: 0.0053
Epoch 5/15
9/9 [==============================] - 8s 880ms/step - loss: 0.4959 - accuracy: 0.7457 - val_loss: 0.6952 - val_accuracy: 0.4954 - lr: 0.0054
Epoch 6/15
9/9 [==============================] - 8s 953ms/step - loss: 0.4933 - accuracy: 0.7400 - val_loss: 0.4439 - val_accuracy: 0.8147 - lr:

2/2 [==============================] - 4s 196ms/step
Exponential : SGD -> mIoU : 74.25%


In [ ]:
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']

plt.figure()
plt.plot( loss, 'r', label='Training loss')
plt.plot( val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

In [ ]:
import random

indx = random.randint(0, X_test.shape[0])
f, axarr = plt.subplots(1,3)
axarr[0].imshow(y_predictions[indx][:,:,1] > threshold)
axarr[1].imshow(y_test[indx])
axarr[2].imshow(X_test[indx])